In [3]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

In [ ]:
## Ignore depots, pretend all starting from Yuma, just have negative demand on the prep sites

In [10]:
## Data
distance = [0]
# prep = columns; carry_in = rows
supply = [0] # prep, len(supply) = len(distance[1])
demand = [0] # carry_in, len(demand) = len(distance)

### Need data where
# distance[I,J] has carry_ins i and prep j
# supply[j] with supply of prep in order J
# demand[i] with demand of carry_ins in order I

num_carry_in = len(demand) ## Carry-in sites are rows of distance matrix
all_carry_in = range(num_carry_in)

num_prep_site = len(supply) ## Prep sites are cols of distance matrix
all_prep = range(num_prep_site)

#if len(supply) < len(distance[1]):
#    print("Not enough supply values defined.")
#    elif len(supply > len(distance[1])):
#        print("Too many supply values defined.")
        

In [5]:
## Model 
# Declare solver       
solver = pywraplp.Solver('SolveAssignmentProblemMIP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Declare binary decision variables
#i by j array segments where (i,j) is 1 if carry_in i is assigned to prep j and 0 otherwise
segments = {}
for i in all_carry_in:
    for j in all_prep:
      segments[(i, j)] = solver.BoolVar('seg[%i,%i]' % (i, j))
    
# Objective: minimize total distance between prep and carry sites 
#does not represent optimal minimization of driving, but ensures proximity
solver.Minimize(solver.Sum([cost[i][j] * segments[i,j] for i in all_carry_in
                                                       for j in all_prep]))

## Constraints
# carry_in i is assigned one and only one prep j
## I think this is creating one sum for each carry_in i (do I need to switch i and j?)
for i in all_carry_in:
    solver.Add(solver.Sum([segments[i, j] for j in all_prep]) == 1)

# sum of demand of carry_ins I assigned to j do not exceed supply of prep j
for j in all_prep:
    solver.Add(solver.Sum([segments[i,j]*demand[i]] for i in all_carry_in) <= supply[j])


In [ ]:
## Results
print('Total cost = ', solver.Objective().Value())

#Prints assignments
for i in all_carry_in:
    for j in all_prep:
      if segments[i, j].solution_value() > 0:
        print('Carry in site %d assigned to prep site %d.' % (i,j))
